In [ ]:
!pip install gymnasium

In [ ]:
import gymnasium as gym
import numpy as np
import time
from IPython import display

In [ ]:
env = gym.make('FrozenLake-v1', render_mode="ansi")

In [ ]:
env.P[0][3] # Transition model

[(0.3333333333333333, 1, 0.0, False),
 (0.3333333333333333, 0, 0.0, False),
 (0.3333333333333333, 0, 0.0, False)]

In [ ]:
env.observation_space.n

16

In [ ]:
env.action_space.n

4

In [ ]:
def play(env, policy, render=False):
    state, _ = env.reset()
    total_reward = 0
    steps = 0
    done = False
    while not done:
        action = policy[state]
        next_state, reward, done, info, _ = env.step(action)
        total_reward += reward
        steps += 1
        if render:
            print(env.render())
            time.sleep(0.5)
            if not done:
                display.clear_output(wait=True)
        state = next_state

    return (total_reward, steps)

In [ ]:
policy_0 = np.asarray([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
play(env, policy_0)

(0.0, 22)

In [ ]:

policy_0 = np.asarray([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
play(env, policy_0, True)


  (Left)
SFFF
FHFH
FFFH
HFFG



(0.0, 12)

In [ ]:
policy_1 = np.asarray([0, 1, 1, 3, 1, 0, 2, 0, 1, 1, 2, 2, 3, 3, 1, 0])
play(env, policy_1, True)

  (Down)
SFFF
FHFH
FFFH
HFFG



(0.0, 4)

In [ ]:
policy_2 = np.array([1, 1, 1, 3, 0, 1, 2, 3, 1, 1, 2, 3, 2, 2, 1, 3])
play(env, policy_2, True)

  (Down)
SFFF
FHFH
FFFH
HFFG



(0.0, 2)

In [ ]:
policy_3 = np.array([0, 3, 0, 3, 0, 0, 0, 0, 3, 1, 0, 0, 0, 2, 1, 0])
play(env, policy_3, True)

  (Down)
SFFF
FHFH
FFFH
HFFG



(1.0, 72)

In [ ]:
def play_multiple_times(env, policy, max_episodes):
    success = 0
    list_of_steps = []
    for i in range(max_episodes):
        total_reward, steps = play(env, policy)

        if total_reward > 0:
            success += 1
            list_of_steps.append(steps)

    print(f'Number of successes: {success}/{max_episodes}')
    print(f'Average number of steps: {np.mean(list_of_steps)}')

In [ ]:
policy_0 = np.asarray([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
play_multiple_times(env, policy_0, 1000)

Number of successes: 0/1000
Average number of steps: nan


In [ ]:
policy_1 = np.asarray([0, 1, 1, 3, 1, 0, 2, 0, 1, 1, 2, 2, 3, 3, 1, 0])
play_multiple_times(env, policy_1, 1000)

Number of successes: 63/1000
Average number of steps: 11.714285714285714


In [ ]:
policy_2 = np.array([1, 1, 1, 3, 0, 1, 2, 3, 1, 1, 2, 3, 2, 2, 1, 3])
play_multiple_times(env, policy_2, 1000)

Number of successes: 100/1000
Average number of steps: 14.17


In [ ]:
policy_3 = np.array([0, 3, 0, 3, 0, 0, 0, 0, 3, 1, 0, 0, 0, 2, 1, 0])
play_multiple_times(env, policy_3, 1000)

Number of successes: 794/1000
Average number of steps: 42.87909319899244


In [ ]:
def policy_evaluation(env, policy, max_iters=500, gamma=0.9):
    # Initialize the values of all states to be 0
    v_values = np.zeros(env.observation_space.n)

    for i in range(max_iters):
        prev_v_values = np.copy(v_values)

        # Update the value of each state
        for state in range(env.observation_space.n):
            action = policy[state]

            # Compute the q-value of the action
            q_value = 0
            for prob, next_state, reward, done in env.P[state][action]:
                q_value += prob * (reward + gamma * prev_v_values[next_state])

            v_values[state] = q_value # update v-value

        # Check convergence
        if np.all(np.isclose(v_values, prev_v_values)):
            print(f'Converged at {i}-th iteration.')
            break

    return v_values

In [ ]:
policy_0 = np.asarray([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
v_values_0 = policy_evaluation(env, policy_0)
print(v_values_0)

Converged at 0-th iteration.
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [ ]:
policy_1 = np.asarray([0, 1, 1, 3, 1, 0, 2, 0, 1, 1, 2, 2, 3, 3, 1, 0])
v_values_1 = policy_evaluation(env, policy_1)
print(v_values_1)

Converged at 48-th iteration.
[0.01904157 0.01519815 0.03161906 0.02371389 0.02538879 0.
 0.06648515 0.         0.05924054 0.13822794 0.18999823 0.
 0.         0.21152109 0.56684236 0.        ]


In [ ]:
np.all(v_values_1 >= v_values_0)

True

In [ ]:
policy_2 = np.array([1, 1, 1, 3, 0, 1, 2, 3, 1, 1, 2, 3, 2, 2, 1, 3])
v_values_2 = policy_evaluation(env, policy_2)
print(v_values_2)

Converged at 53-th iteration.
[0.02889625 0.01951972 0.03616977 0.0271268  0.04790519 0.
 0.07391985 0.         0.08288277 0.19339319 0.21022995 0.
 0.         0.35153135 0.62684674 0.        ]


In [ ]:
np.all(v_values_2 >= v_values_1)

True

In [ ]:
policy_3 = np.array([0, 3, 0, 3, 0, 0, 0, 0, 3, 1, 0, 0, 0, 2, 1, 0])
v_values_3 = policy_evaluation(env, policy_3)
print(v_values_3)

Converged at 80-th iteration.
[0.06888666 0.06141097 0.07440714 0.05580443 0.09185068 0.
 0.11220679 0.         0.14543323 0.24749485 0.29961611 0.
 0.         0.37993438 0.63901935 0.        ]


In [ ]:
np.all(v_values_3 >= v_values_2)

True

In [ ]:
def value_iteration(env, max_iters=500, gamma=0.9):
    # initialize
    v_values = np.zeros(env.observation_space.n)

    for i in range(max_iters):
        prev_v_values = np.copy(v_values)

        # update the v-value for each state
        for state in range(env.observation_space.n):
            q_values = []

            # compute the q-value for each action that we can perform at the state
            for action in range(env.action_space.n):
                q_value = 0
                # loop through each possible outcome
                for prob, next_state, reward, done in env.P[state][action]:
                    q_value += prob * (reward + gamma * prev_v_values[next_state])

                q_values.append(q_value)

            # select the max q-values
            best_action = np.argmax(q_values)
            v_values[state] = q_values[best_action]

        # check convergence
        if np.all(np.isclose(v_values, prev_v_values)):
            print(f'Converged at {i}-th iteration.')
            break

    return v_values

In [ ]:
optimal_v_values = value_iteration(env, max_iters=500, gamma=0.9)

Converged at 79-th iteration.


In [ ]:
optimal_v_values

array([0.06888615, 0.06141054, 0.07440682, 0.05580409, 0.09185022,
       0.        , 0.11220663, 0.        , 0.14543286, 0.2474946 ,
       0.29961593, 0.        , 0.        , 0.3799342 , 0.63901926,
       0.        ])

In [ ]:
def policy_extraction(env, v_values, gamma=0.9):
    # initialize
    policy = np.zeros(env.observation_space.n, dtype=np.int32)

    # loop through each state in the environment
    for state in range(env.observation_space.n):
        q_values = []
        # loop through each action
        for action in range(env.action_space.n):
            q_value = 0
            # loop each possible outcome
            for prob, next_state, reward, done in env.P[state][action]:
                q_value += prob * (reward + gamma * v_values[next_state])

            q_values.append(q_value)

        # select the best action
        best_action = np.argmax(q_values)
        policy[state] = best_action

    return policy

In [ ]:
optimal_policy = policy_extraction(env, optimal_v_values, gamma=0.9)

In [ ]:
optimal_policy

array([0, 3, 0, 3, 0, 0, 0, 0, 3, 1, 0, 0, 0, 2, 1, 0], dtype=int32)

In [ ]:
play(env, optimal_policy, True)

  (Left)
SFFF
FHFH
FFFH
HFFG



(0.0, 23)

In [ ]:
play_multiple_times(env, optimal_policy, 1000)

Number of successes: 801/1000
Average number of steps: 44.17977528089887


In [ ]:
def policy_iteration(env, max_iters=500, gamma=0.9):
    # Initialize the policy randomly
    policy = np.random.randint(0, env.action_space.n, env.observation_space.n)

    for i in range(max_iters):
        # Policy Evaluation
        v_values = policy_evaluation(env, policy, max_iters, gamma)

        policy_stable = True

        # Policy Improvement
        for state in range(env.observation_space.n):
            old_action = policy[state]

            q_values = np.zeros(env.action_space.n)

            # Compute q-value for each action
            for action in range(env.action_space.n):
                for prob, next_state, reward, done in env.P[state][action]:
                    q_values[action] += prob * (reward + gamma * v_values[next_state])

            # Select best action
            best_action = np.argmax(q_values)
            policy[state] = best_action

            # Check if policy is stable
            if old_action != best_action:
                policy_stable = False

        if policy_stable:
            print(f'Converged at {i+1}-th iteration.')
            break

    return v_values, policy

#FrozenLake-v1

## Value Iteration

In [ ]:
optimal_v_values = value_iteration(env, max_iters=500, gamma=0.9)
optimal_policy = policy_extraction(env, optimal_v_values, gamma=0.9)

Converged at 79-th iteration.


In [ ]:
optimal_v_values

array([0.06888615, 0.06141054, 0.07440682, 0.05580409, 0.09185022,
       0.        , 0.11220663, 0.        , 0.14543286, 0.2474946 ,
       0.29961593, 0.        , 0.        , 0.3799342 , 0.63901926,
       0.        ])

In [ ]:
optimal_policy

array([0, 3, 0, 3, 0, 0, 0, 0, 3, 1, 0, 0, 0, 2, 1, 0], dtype=int32)

In [ ]:
play(env, optimal_policy, True)

  (Down)
SFFF
FHFH
FFFH
HFFG



(1.0, 63)

In [ ]:
play_multiple_times(env, optimal_policy, 1000)

Number of successes: 792/1000
Average number of steps: 43.916666666666664


In [ ]:
num_runs = 100

value_iteration_total_time = 0

for _ in range(num_runs):
    start_time = time.time()
    optimal_v_values = value_iteration(env, max_iters=500, gamma=0.9)
    policy_extraction(env, optimal_v_values, gamma=0.9)
    end_time = time.time()
    value_iteration_total_time += end_time - start_time

value_iteration_avg_time = value_iteration_total_time / num_runs
value_iteration_avg_time

Converged at 79-th iteration.
Converged at 79-th iteration.
Converged at 79-th iteration.
Converged at 79-th iteration.
Converged at 79-th iteration.
Converged at 79-th iteration.
Converged at 79-th iteration.
Converged at 79-th iteration.
Converged at 79-th iteration.
Converged at 79-th iteration.
Converged at 79-th iteration.
Converged at 79-th iteration.
Converged at 79-th iteration.
Converged at 79-th iteration.
Converged at 79-th iteration.
Converged at 79-th iteration.
Converged at 79-th iteration.
Converged at 79-th iteration.
Converged at 79-th iteration.
Converged at 79-th iteration.
Converged at 79-th iteration.
Converged at 79-th iteration.
Converged at 79-th iteration.
Converged at 79-th iteration.
Converged at 79-th iteration.
Converged at 79-th iteration.
Converged at 79-th iteration.
Converged at 79-th iteration.
Converged at 79-th iteration.
Converged at 79-th iteration.
Converged at 79-th iteration.
Converged at 79-th iteration.
Converged at 79-th iteration.
Converged 

0.12130724430084229

## Policy Iteration

In [ ]:
optimal_v_values, optimal_policy = policy_iteration(env, max_iters=500, gamma=0.9)

Converged at 30-th iteration.
Converged at 78-th iteration.
Converged at 79-th iteration.
Converged at 80-th iteration.
Converged at 4-th iteration.


In [ ]:
optimal_v_values

array([0.06888666, 0.06141097, 0.07440714, 0.05580443, 0.09185068,
       0.        , 0.11220679, 0.        , 0.14543323, 0.24749485,
       0.29961611, 0.        , 0.        , 0.37993438, 0.63901935,
       0.        ])

In [ ]:
optimal_policy

array([0, 3, 0, 3, 0, 0, 0, 0, 3, 1, 0, 0, 0, 2, 1, 0])

In [ ]:
play(env, optimal_policy, True)

  (Down)
SFFF
FHFH
FFFH
HFFG



(1.0, 46)

In [ ]:
play_multiple_times(env, optimal_policy, 1000)

Number of successes: 791/1000
Average number of steps: 43.30341340075854


In [ ]:
num_runs = 100

policy_iteration_total_time = 0

for _ in range(num_runs):
    start_time = time.time()
    policy_iteration(env, max_iters=500, gamma=0.9)
    end_time = time.time()
    policy_iteration_total_time += end_time - start_time

policy_iteration_avg_time = policy_iteration_total_time / num_runs
policy_iteration_avg_time

Converged at 25-th iteration.
Converged at 58-th iteration.
Converged at 79-th iteration.
Converged at 80-th iteration.
Converged at 4-th iteration.
Converged at 36-th iteration.
Converged at 61-th iteration.
Converged at 79-th iteration.
Converged at 80-th iteration.
Converged at 4-th iteration.
Converged at 39-th iteration.
Converged at 77-th iteration.
Converged at 74-th iteration.
Converged at 80-th iteration.
Converged at 4-th iteration.
Converged at 0-th iteration.
Converged at 23-th iteration.
Converged at 59-th iteration.
Converged at 62-th iteration.
Converged at 79-th iteration.
Converged at 80-th iteration.
Converged at 6-th iteration.
Converged at 0-th iteration.
Converged at 23-th iteration.
Converged at 59-th iteration.
Converged at 62-th iteration.
Converged at 79-th iteration.
Converged at 80-th iteration.
Converged at 6-th iteration.
Converged at 66-th iteration.
Converged at 74-th iteration.
Converged at 80-th iteration.
Converged at 3-th iteration.
Converged at 35-th

0.07989896059036256

# FrozenLake8x8-v1

In [ ]:
env = gym.make('FrozenLake8x8-v1')

## Value Iteration

In [ ]:
optimal_v_values = value_iteration(env, max_iters=500, gamma=0.9)
optimal_policy = policy_extraction(env, optimal_v_values, gamma=0.9)

Converged at 117-th iteration.


In [ ]:
optimal_v_values

array([0.00641104, 0.00854808, 0.01230044, 0.01778942, 0.02508214,
       0.03247089, 0.03957134, 0.04297844, 0.00602405, 0.00764512,
       0.01091162, 0.01642654, 0.02605411, 0.03619409, 0.0493547 ,
       0.05730461, 0.00509024, 0.0058532 , 0.00677534, 0.        ,
       0.02557084, 0.03882139, 0.06763973, 0.08435607, 0.0042256 ,
       0.00476954, 0.00581968, 0.0078541 , 0.02036065, 0.        ,
       0.09175501, 0.12919111, 0.00318093, 0.00319659, 0.00270488,
       0.        , 0.0344439 , 0.06195145, 0.10901921, 0.20969093,
       0.00186915, 0.        , 0.        , 0.01085079, 0.03250092,
       0.06304172, 0.        , 0.36008773, 0.00118046, 0.        ,
       0.00137717, 0.00366839, 0.        , 0.11568671, 0.        ,
       0.63051379, 0.00088531, 0.00077462, 0.00092218, 0.        ,
       0.13824885, 0.32258065, 0.61443932, 0.        ])

In [ ]:
optimal_policy

array([3, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 2, 2, 2, 1, 3, 3, 0, 0, 2, 3,
       2, 1, 3, 3, 3, 1, 0, 0, 2, 1, 3, 3, 0, 0, 2, 1, 3, 2, 0, 0, 0, 1,
       3, 0, 0, 2, 0, 0, 1, 0, 0, 0, 0, 2, 0, 1, 0, 0, 1, 1, 1, 0],
      dtype=int32)

In [ ]:
play(env, optimal_policy, True)

None


(1.0, 85)

In [ ]:
play_multiple_times(env, optimal_policy, 1000)

Number of successes: 764/1000
Average number of steps: 75.68062827225131


In [ ]:
num_runs = 100

value_iteration_total_time = 0

for _ in range(num_runs):
    start_time = time.time()
    optimal_v_values = value_iteration(env, max_iters=500, gamma=0.9)
    policy_extraction(env, optimal_v_values, gamma=0.9)
    end_time = time.time()
    value_iteration_total_time += end_time - start_time

value_iteration_avg_time = value_iteration_total_time / num_runs
value_iteration_avg_time

Converged at 117-th iteration.
Converged at 117-th iteration.
Converged at 117-th iteration.
Converged at 117-th iteration.
Converged at 117-th iteration.
Converged at 117-th iteration.
Converged at 117-th iteration.
Converged at 117-th iteration.
Converged at 117-th iteration.
Converged at 117-th iteration.
Converged at 117-th iteration.
Converged at 117-th iteration.
Converged at 117-th iteration.
Converged at 117-th iteration.
Converged at 117-th iteration.
Converged at 117-th iteration.
Converged at 117-th iteration.
Converged at 117-th iteration.
Converged at 117-th iteration.
Converged at 117-th iteration.
Converged at 117-th iteration.
Converged at 117-th iteration.
Converged at 117-th iteration.
Converged at 117-th iteration.
Converged at 117-th iteration.
Converged at 117-th iteration.
Converged at 117-th iteration.
Converged at 117-th iteration.
Converged at 117-th iteration.
Converged at 117-th iteration.
Converged at 117-th iteration.
Converged at 117-th iteration.
Converge

0.6623348569869996

## Policy Iteration

In [ ]:
optimal_v_values, optimal_policy = policy_iteration(env, max_iters=500, gamma=0.9)

Converged at 32-th iteration.
Converged at 43-th iteration.
Converged at 95-th iteration.
Converged at 92-th iteration.
Converged at 95-th iteration.
Converged at 103-th iteration.
Converged at 110-th iteration.
Converged at 117-th iteration.
Converged at 8-th iteration.


In [ ]:
optimal_v_values

array([0.00641104, 0.00854808, 0.01230044, 0.01778942, 0.02508214,
       0.03247089, 0.03957134, 0.04297844, 0.00602405, 0.00764512,
       0.01091162, 0.01642654, 0.02605411, 0.03619409, 0.0493547 ,
       0.05730461, 0.00509024, 0.0058532 , 0.00677534, 0.        ,
       0.02557084, 0.03882139, 0.06763973, 0.08435607, 0.0042256 ,
       0.00476954, 0.00581968, 0.0078541 , 0.02036065, 0.        ,
       0.09175501, 0.12919111, 0.00318093, 0.00319659, 0.00270488,
       0.        , 0.0344439 , 0.06195145, 0.10901921, 0.20969093,
       0.00186915, 0.        , 0.        , 0.01085079, 0.03250092,
       0.06304172, 0.        , 0.36008773, 0.00118046, 0.        ,
       0.00137717, 0.00366839, 0.        , 0.11568671, 0.        ,
       0.63051379, 0.0008853 , 0.00077462, 0.00092218, 0.        ,
       0.13824885, 0.32258065, 0.61443932, 0.        ])

In [ ]:
optimal_policy

array([3, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 2, 2, 2, 1, 3, 3, 0, 0, 2, 3,
       2, 1, 3, 3, 3, 1, 0, 0, 2, 1, 3, 3, 0, 0, 2, 1, 3, 2, 0, 0, 0, 1,
       3, 0, 0, 2, 0, 0, 1, 0, 0, 0, 0, 2, 0, 1, 0, 0, 1, 1, 1, 0])

In [ ]:
play(env, optimal_policy, True)

None


(1.0, 28)

In [ ]:
play_multiple_times(env, optimal_policy, 1000)

Number of successes: 746/1000
Average number of steps: 75.78686327077747


In [ ]:
num_runs = 100

policy_iteration_total_time = 0

for _ in range(num_runs):
    start_time = time.time()
    policy_iteration(env, max_iters=500, gamma=0.9)
    end_time = time.time()
    policy_iteration_total_time += end_time - start_time

policy_iteration_avg_time = policy_iteration_total_time / num_runs
policy_iteration_avg_time

Converged at 58-th iteration.
Converged at 97-th iteration.
Converged at 108-th iteration.
Converged at 111-th iteration.
Converged at 117-th iteration.
Converged at 5-th iteration.
Converged at 44-th iteration.
Converged at 114-th iteration.
Converged at 117-th iteration.
Converged at 3-th iteration.
Converged at 10-th iteration.
Converged at 32-th iteration.
Converged at 92-th iteration.
Converged at 90-th iteration.
Converged at 92-th iteration.
Converged at 95-th iteration.
Converged at 100-th iteration.
Converged at 112-th iteration.
Converged at 117-th iteration.
Converged at 9-th iteration.
Converged at 0-th iteration.
Converged at 27-th iteration.
Converged at 91-th iteration.
Converged at 92-th iteration.
Converged at 90-th iteration.
Converged at 92-th iteration.
Converged at 95-th iteration.
Converged at 100-th iteration.
Converged at 112-th iteration.
Converged at 117-th iteration.
Converged at 10-th iteration.
Converged at 76-th iteration.
Converged at 119-th iteration.
Co

0.596421115398407

# Taxi-v3

In [ ]:
env = gym.make('Taxi-v3')

## Value Iteration

In [ ]:
optimal_v_values = value_iteration(env, max_iters=500, gamma=0.9)
optimal_policy = policy_extraction(env, optimal_v_values, gamma=0.9)

Converged at 116-th iteration.


In [ ]:
optimal_v_values

array([ 89.47323891,  32.81971401,  55.26423891,  37.57755845,
         8.43222921,  32.81971401,   8.43222921,  15.28447953,
        32.81971401,  18.09386122,  55.26423891,  21.2154998 ,
        12.75594298,  18.09386122,  12.75594298,  37.57755845,
       100.52591945,  37.57755845,  62.51591945,  42.86394891,
        79.52591945,  28.53774704,  48.73781945,  32.81971401,
        10.48035311,  37.57755845,  10.48035311,  18.09386122,
        28.53774704,  15.28447953,  48.73781945,  18.09386122,
        15.28447953,  21.2154998 ,  15.28447953,  42.86394891,
        89.47323891,  42.86394891,  55.26423891,  48.73781945,
        42.86394891,  12.75594298,  24.68388374,  15.28447953,
        24.68388374,  70.57323891,  24.68388374,  37.57755845,
        24.68388374,  12.75594298,  42.86394891,  15.28447953,
        18.09386122,  24.68388374,  18.09386122,  48.73781945,
        48.73781945,  79.52591945,  48.73781945,  55.26423891,
        37.57755845,  10.48035311,  21.2154998 ,  12.75

In [ ]:
optimal_policy

array([4, 4, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 3, 3,
       3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0,
       2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 2, 2,
       2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 4, 4, 4, 4,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 0, 0, 0, 0, 2, 2, 2, 2, 1, 2, 0, 2,
       1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 2, 2, 2, 2, 1, 2, 3, 2, 3, 3,
       3, 3, 1, 1, 1, 1, 3, 3, 3, 3, 2, 2, 2, 2, 3, 1, 3, 2, 3, 3, 3, 3,
       1, 1, 1, 1, 3, 3, 3, 3, 0, 0, 0, 0, 3, 1, 3, 0, 3, 3, 3, 3, 1, 1,
       1, 1, 3, 3, 3, 3, 0, 0, 0, 0, 3, 1, 3, 0, 1,

In [ ]:
play(env, optimal_policy, True)

None


(7, 14)

In [ ]:
play_multiple_times(env, optimal_policy, 1000)

Number of successes: 1000/1000
Average number of steps: 13.018


In [ ]:
num_runs = 100

value_iteration_total_time = 0

for _ in range(num_runs):
    start_time = time.time()
    optimal_v_values = value_iteration(env, max_iters=500, gamma=0.9)
    policy_extraction(env, optimal_v_values, gamma=0.9)
    end_time = time.time()
    value_iteration_total_time += end_time - start_time

value_iteration_avg_time = value_iteration_total_time / num_runs
value_iteration_avg_time

Converged at 116-th iteration.
Converged at 116-th iteration.
Converged at 116-th iteration.
Converged at 116-th iteration.
Converged at 116-th iteration.
Converged at 116-th iteration.
Converged at 116-th iteration.
Converged at 116-th iteration.
Converged at 116-th iteration.
Converged at 116-th iteration.
Converged at 116-th iteration.
Converged at 116-th iteration.
Converged at 116-th iteration.
Converged at 116-th iteration.
Converged at 116-th iteration.
Converged at 116-th iteration.
Converged at 116-th iteration.
Converged at 116-th iteration.
Converged at 116-th iteration.
Converged at 116-th iteration.
Converged at 116-th iteration.
Converged at 116-th iteration.
Converged at 116-th iteration.
Converged at 116-th iteration.
Converged at 116-th iteration.
Converged at 116-th iteration.
Converged at 116-th iteration.
Converged at 116-th iteration.
Converged at 116-th iteration.
Converged at 116-th iteration.
Converged at 116-th iteration.
Converged at 116-th iteration.
Converge

6.230894536972046

## Policy Iteration

In [ ]:
optimal_v_values, optimal_policy = policy_iteration(env, max_iters=500, gamma=0.9)

Converged at 94-th iteration.
Converged at 98-th iteration.
Converged at 100-th iteration.
Converged at 102-th iteration.
Converged at 103-th iteration.
Converged at 104-th iteration.
Converged at 106-th iteration.
Converged at 109-th iteration.
Converged at 110-th iteration.
Converged at 111-th iteration.
Converged at 112-th iteration.
Converged at 115-th iteration.
Converged at 116-th iteration.
Converged at 116-th iteration.
Converged at 116-th iteration.
Converged at 116-th iteration.
Converged at 116-th iteration.
Converged at 116-th iteration.
Converged at 18-th iteration.


In [ ]:
optimal_v_values

array([ 89.47323891,  32.81971401,  55.26423891,  37.57755845,
         8.43222921,  32.81971401,   8.43222921,  15.28447953,
        32.81971401,  18.09386122,  55.26423891,  21.2154998 ,
        12.75594298,  18.09386122,  12.75594298,  37.57755845,
       100.52591945,  37.57755845,  62.51591945,  42.86394891,
        79.52591945,  28.53774704,  48.73781945,  32.81971401,
        10.48035311,  37.57755845,  10.48035311,  18.09386122,
        28.53774704,  15.28447953,  48.73781945,  18.09386122,
        15.28447953,  21.2154998 ,  15.28447953,  42.86394891,
        89.47323891,  42.86394891,  55.26423891,  48.73781945,
        42.86394891,  12.75594298,  24.68388374,  15.28447953,
        24.68388374,  70.57323891,  24.68388374,  37.57755845,
        24.68388374,  12.75594298,  42.86394891,  15.28447953,
        18.09386122,  24.68388374,  18.09386122,  48.73781945,
        48.73781945,  79.52591945,  48.73781945,  55.26423891,
        37.57755845,  10.48035311,  21.2154998 ,  12.75

In [ ]:
optimal_policy

array([4, 4, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 3, 3,
       3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0,
       2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 2, 2,
       2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 4, 4, 4, 4,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 0, 0, 0, 0, 2, 2, 2, 2, 1, 2, 0, 2,
       1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 2, 2, 2, 2, 1, 2, 3, 2, 3, 3,
       3, 3, 1, 1, 1, 1, 3, 3, 3, 3, 2, 2, 2, 2, 3, 1, 3, 2, 3, 3, 3, 3,
       1, 1, 1, 1, 3, 3, 3, 3, 0, 0, 0, 0, 3, 1, 3, 0, 3, 3, 3, 3, 1, 1,
       1, 1, 3, 3, 3, 3, 0, 0, 0, 0, 3, 1, 3, 0, 1,

In [ ]:
play(env, optimal_policy, True)

None


(7, 14)

In [ ]:
play_multiple_times(env, optimal_policy, 1000)

Number of successes: 1000/1000
Average number of steps: 13.028


In [ ]:
num_runs = 100

policy_iteration_total_time = 0

for _ in range(num_runs):
    start_time = time.time()
    policy_iteration(env, max_iters=500, gamma=0.9)
    end_time = time.time()
    policy_iteration_total_time += end_time - start_time

policy_iteration_avg_time = policy_iteration_total_time / num_runs
policy_iteration_avg_time

Converged at 97-th iteration.
Converged at 98-th iteration.
Converged at 101-th iteration.
Converged at 101-th iteration.
Converged at 102-th iteration.
Converged at 104-th iteration.
Converged at 105-th iteration.
Converged at 106-th iteration.
Converged at 110-th iteration.
Converged at 111-th iteration.
Converged at 111-th iteration.
Converged at 112-th iteration.
Converged at 115-th iteration.
Converged at 116-th iteration.
Converged at 116-th iteration.
Converged at 116-th iteration.
Converged at 116-th iteration.
Converged at 116-th iteration.
Converged at 18-th iteration.
Converged at 91-th iteration.
Converged at 89-th iteration.
Converged at 100-th iteration.
Converged at 101-th iteration.
Converged at 102-th iteration.
Converged at 104-th iteration.
Converged at 105-th iteration.
Converged at 106-th iteration.
Converged at 109-th iteration.
Converged at 110-th iteration.
Converged at 111-th iteration.
Converged at 112-th iteration.
Converged at 115-th iteration.
Converged at 

13.150977292060851

# Summary

Đối với các toy games 'FrozenLake-v1', 'FrozenLake8x8-v1' và 'Taxi-v3', có thể có những nhận xét và so sánh giữa Policy Iteration và Value Iteration như sau:


*   FrozenLake-v1 và FrozenLake8x8-v1:

  + Policy Iteration và Value Iteration có kết quả trận thắng và bước đi trung bình tương tương nên hiệu suất của chúng bằng nhau.
  + Policy Iteration thường hội tụ nhanh hơn so với Value Iteration trong các môi trường này vì Policy Iteration cần ít lần lặp hơn để đạt được sự hội tụ. Điều này là do Policy Iteration thực hiện đánh giá và cải thiện chúng một cách lần lượt và có thể dễ dàng hội tụ khi Policy Iteration được cải thiện đến mức tối ưu còn Value Iteration có thể mất nhiều thời gian hơn để hội tụ trong các môi trường FrozenLake-v1 và FrozenLake8x8-v1 vì nó cần cập nhật giá trị của tất cả các trạng thái ở mỗi lần lặp.

*   Taxi-v3:

  + Policy Iteration và Value Iteration có kết quả trận thắng và bước đi trung bình tương tương nên hiệu suất của chúng bằng nhau.
  + Value Iteration có vẻ nhỉnh hơn so với Policy Iteration trong các môi trường này vì thời gian chạy nhanh hơn so với Policy Iteration.